In [1]:
from __future__ import print_function

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("extraccion de datos del censo") \
    .getOrCreate()

from pyspark.sql.functions import col, filter

24/03/30 20:06:50 WARN Utils: Your hostname, onur-ideacenter resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlo1)
24/03/30 20:06:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/30 20:06:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc = spark.sparkContext
sc.setLogLevel("WARN")

#path = '/media/DiscoLocal/BigData/spark/'
#path = 'file:///home/sparkuser/'
path = 'data/'

# Lee el fichero de microdatos
rdd = spark.sparkContext.textFile(path + 'censo2011s.txt.bz2')

# ¿cuántas particiones tenemos?
print("Partitions:", rdd.getNumPartitions())

Partitions: 2


In [3]:
import cparser
sc.addPyFile('cparser.py')
parsed = rdd.map(cparser.field_split)

from pyspark.sql import Row
# Definimos el objeto de fila usando los nombres de columna
CensoRow = Row(*cparser.NAMES._fields)
# Mapeamos cada fila a un objeto de fila
rows = parsed.map(lambda x: CensoRow(*x))

# RDD de Row a DataFrame
censo = spark.createDataFrame(rows)

print("Schema:")
censo.printSchema()

print("Size:", censo.count())

Schema:
root
 |-- CPRO: string (nullable = true)
 |-- CMUN: string (nullable = true)
 |-- IDHUECO: string (nullable = true)
 |-- NORDEN: string (nullable = true)
 |-- FACTOR: string (nullable = true)
 |-- MNAC: string (nullable = true)
 |-- ANAC: string (nullable = true)
 |-- EDAD: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- NACI: string (nullable = true)
 |-- CPAISN: string (nullable = true)
 |-- CPRON: string (nullable = true)
 |-- CMUNN: string (nullable = true)
 |-- ANORES: string (nullable = true)
 |-- ANOM: string (nullable = true)
 |-- ANOC: string (nullable = true)
 |-- ANOE: string (nullable = true)
 |-- CLPAIS: string (nullable = true)
 |-- CLPRO: string (nullable = true)
 |-- CLMUNP: string (nullable = true)
 |-- RES_ANTERIOR: string (nullable = true)
 |-- CPAISUNANO: string (nullable = true)
 |-- CPROUNANO: string (nullable = true)
 |-- CMUNANO: string (nullable = true)
 |-- RES_UNANO: string (nullable = true)
 |-- CPAISDANO: string (nullable = true)
 

24/03/30 20:09:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Size: 410605


In [4]:
# Carga los nombres de municipio
cmun = spark.read.format('csv').option("header", True).option("encoding", "utf-8").load(path + 'codes-cmun.csv')

# Añade el nombre del municipio de residencia al Dataframe
# Hacemos un left join para no perder los registros que (por privacidad)
# no contienen información de municipio
cond = [censo.CMUN == cmun.CMUN, censo.CPRO == cmun.CPRO]
censo_mun = censo.join(cmun, on=cond, how='left').drop(cmun.CPRO).drop(cmun.CMUN).drop(cmun.DC)

print("Schema + MUN:")
censo_mun.printSchema()

print("Sample:")
for row in censo_mun.limit(3).collect():
    print(row)

Schema + MUN:
root
 |-- CPRO: string (nullable = true)
 |-- CMUN: string (nullable = true)
 |-- IDHUECO: string (nullable = true)
 |-- NORDEN: string (nullable = true)
 |-- FACTOR: string (nullable = true)
 |-- MNAC: string (nullable = true)
 |-- ANAC: string (nullable = true)
 |-- EDAD: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- NACI: string (nullable = true)
 |-- CPAISN: string (nullable = true)
 |-- CPRON: string (nullable = true)
 |-- CMUNN: string (nullable = true)
 |-- ANORES: string (nullable = true)
 |-- ANOM: string (nullable = true)
 |-- ANOC: string (nullable = true)
 |-- ANOE: string (nullable = true)
 |-- CLPAIS: string (nullable = true)
 |-- CLPRO: string (nullable = true)
 |-- CLMUNP: string (nullable = true)
 |-- RES_ANTERIOR: string (nullable = true)
 |-- CPAISUNANO: string (nullable = true)
 |-- CPROUNANO: string (nullable = true)
 |-- CMUNANO: string (nullable = true)
 |-- RES_UNANO: string (nullable = true)
 |-- CPAISDANO: string (nullable = t

Row(CPRO='01', CMUN='059', IDHUECO='0000000356', NORDEN='0002', FACTOR='011.0002419100', MNAC='02', ANAC='1951', EDAD='060', SEXO='1', NACI='108', CPAISN='108', CPRON='31', CMUNN='992', ANORES='1983', ANOM='1974', ANOC='1974', ANOE='1951', CLPAIS='108', CLPRO='31', CLMUNP='992', RES_ANTERIOR=' ', CPAISUNANO='108', CPROUNANO='01', CMUNANO='059', RES_UNANO='1', CPAISDANO='108', CPRODANO='01', CMUNDANO='059', RES_DANO='1', SEG_VIV='1', SEG_PAIS='108', SEG_PROV='31', SEG_MUN='991', SEG_NOCHES='150', SEG_DISP='1', ECIVIL='2', ESCOLAR=' ', ESREAL='07', TESTUD='5', TAREA1='2', TAREA2='2', TAREA3='2', TAREA4='2', HIJOS=' ', NHIJOS='  ', RELA='5', JORNADA=' ', CNO='  ', CNAE='  ', SITU=' ', CSE='     ', ESCUR1='  ', ESCUR2='  ', ESCUR3='  ', LTRABA=' ', PAISTRABA='   ', PROTRABA='  ', MUNTRABA='   ', CODTRABA='     ', NVIAJE=' ', MDESP1=' ', MDESP2=' ', TDESP=' ', TENEN='1', CALE='2', ASEO='1', BADUCH='1', INTERNET='1', AGUACOR='1', SUT='090', NHAB='06', PLANTAS='08', PLANTAB='0', TIPOEDIF='1',

24/03/30 20:11:16 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 6 (TID 6): Attempting to kill Python Worker


In [5]:
# Repartition DataFrame to have only one partition
censo_mun_single_partition = censo_mun.repartition(1)

# Save DataFrame as a single Parquet file
censo_mun_single_partition.write.format('parquet').mode('overwrite').save(path + 'censo-2011s.parquet')

### 1. Leemos los datos que fueron grabados en Parquet

In [6]:
# Read Parquet file
censo_parquet = spark.read.format('parquet').load(path + 'censo-2011s.parquet')

print("Schema for Parquet File :")
censo_parquet.printSchema()

print("Sample for Parquet File:")
for row in censo_parquet.limit(3).collect():
    print(row)


Schema for Parquet File :
root
 |-- CPRO: string (nullable = true)
 |-- CMUN: string (nullable = true)
 |-- IDHUECO: string (nullable = true)
 |-- NORDEN: string (nullable = true)
 |-- FACTOR: string (nullable = true)
 |-- MNAC: string (nullable = true)
 |-- ANAC: string (nullable = true)
 |-- EDAD: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- NACI: string (nullable = true)
 |-- CPAISN: string (nullable = true)
 |-- CPRON: string (nullable = true)
 |-- CMUNN: string (nullable = true)
 |-- ANORES: string (nullable = true)
 |-- ANOM: string (nullable = true)
 |-- ANOC: string (nullable = true)
 |-- ANOE: string (nullable = true)
 |-- CLPAIS: string (nullable = true)
 |-- CLPRO: string (nullable = true)
 |-- CLMUNP: string (nullable = true)
 |-- RES_ANTERIOR: string (nullable = true)
 |-- CPAISUNANO: string (nullable = true)
 |-- CPROUNANO: string (nullable = true)
 |-- CMUNANO: string (nullable = true)
 |-- RES_UNANO: string (nullable = true)
 |-- CPAISDANO: string (

In [8]:
# Size of the Parquet file (number of rows and columns)
print("Size of Parquet File:", censo_parquet.count())
print("Number of columns in Parquet File:", len(censo_parquet.columns))

Size of Parquet File: 410605
Number of columns in Parquet File: 143


In [9]:
# See the number of partitions in the Parquet file
print("Number of partitions in Parquet File:", censo_parquet.rdd.getNumPartitions())

# See the first 10 rows of the Parquet file
print("First 10 rows of Parquet File:")
censo_parquet.show(10)

Number of partitions in Parquet File: 7
First 10 rows of Parquet File:
+----+----+----------+------+--------------+----+----+----+----+----+------+-----+-----+------+----+----+----+------+-----+------+------------+----------+---------+-------+---------+---------+--------+--------+--------+-------+--------+--------+-------+----------+--------+------+-------+------+------+------+------+------+------+-----+------+----+-------+---+----+----+-----+------+------+------+------+---------+--------+--------+--------+------+------+------+-----+-----+----+----+------+--------+-------+---+----+-------+-------+--------+-------+------+--------+-------+------+------+---+-----+----+-----+-------+----------+----------+----------+----------+-------+------+-----+----+----+------+------+-------+----+-------+-----+-----+---+-----+-----+-----+-----+-----+----+-------+-------+-------+-------+--------+-------+----------+-------+-------+-------+-------+-------+-------+----------+-------+-------+-------+-------+

Realizamos un join para obtener los nombres de las provincias.

In [10]:
# Read the csv file
censo_pro = spark.read.format('csv').option("header", True).option("encoding", "utf-8").load(path + 'codes-prov.csv')

print("Schema for CSV File:")
censo_pro.printSchema()

print("First 10 rows of CSV Province File:")
censo_pro.show(10)

Schema for CSV File:
root
 |-- CPRO: string (nullable = true)
 |-- NOMBRE_PRO: string (nullable = true)

First 10 rows of CSV File:
+----+----------+
|CPRO|NOMBRE_PRO|
+----+----------+
|  01|     Alava|
|  02|  Albacete|
|  03|  Alicante|
|  04|   Almería|
|  05|     Avila|
|  06|   Badajoz|
|  07|  Baleares|
|  08| Barcelona|
|  09|    Burgos|
|  10|   Cáceres|
+----+----------+
only showing top 10 rows



In [11]:
# Perform a join operation between the Parquet file and the CSV file if the column CPRO is the same
censo_data = censo_parquet.join(censo_pro, censo_parquet.CPRO == censo_pro.CPRO, how='left').drop(censo_pro.CPRO)

print("First 10 rows of the joined DataFrame:")
censo_data.show(10)

First 10 rows of the joined DataFrame:
+----+----+----------+------+--------------+----+----+----+----+----+------+-----+-----+------+----+----+----+------+-----+------+------------+----------+---------+-------+---------+---------+--------+--------+--------+-------+--------+--------+-------+----------+--------+------+-------+------+------+------+------+------+------+-----+------+----+-------+---+----+----+-----+------+------+------+------+---------+--------+--------+--------+------+------+------+-----+-----+----+----+------+--------+-------+---+----+-------+-------+--------+-------+------+--------+-------+------+------+---+-----+----+-----+-------+----------+----------+----------+----------+-------+------+-----+----+----+------+------+-------+----+-------+-----+-----+---+-----+-----+-----+-----+-----+----+-------+-------+-------+-------+--------+-------+----------+-------+-------+-------+-------+-------+-------+----------+-------+-------+-------+-------+-------+-------+------+-------+-

### 2. Realizamos un filtrado de las muestras, dejando sólo las de las personas cuyo país de nacimiento sea o Francia o Portugal

In [12]:
# Filter the censo_data by leaving only those of persons whose country of birth is either France or Portugal (CPAISN=110 OR CPAISN=123) with the filter function of the pyspark
censo_filtered = censo_data.filter((censo_data.CPAISN == 110) | (censo_data.CPAISN == 123))

print("First 10 rows of the filtered DataFrame:")
censo_filtered.show(10)

First 10 rows of the filtered DataFrame:
+----+----+----------+------+--------------+----+----+----+----+----+------+-----+-----+------+----+----+----+------+-----+------+------------+----------+---------+-------+---------+---------+--------+--------+--------+-------+--------+--------+-------+----------+--------+------+-------+------+------+------+------+------+------+-----+------+----+-------+---+----+----+-----+------+------+------+------+---------+--------+--------+--------+------+------+------+-----+-----+----+----+------+--------+-------+---+----+-------+-------+--------+-------+------+--------+-------+------+------+---+-----+----+-----+-------+----------+----------+----------+----------+-------+------+-----+----+----+------+------+-------+----+-------+-----+-----+---+-----+-----+-----+-----+-----+----+-------+-------+-------+-------+--------+-------+----------+-------+-------+-------+-------+-------+-------+----------+-------+-------+-------+-------+-------+-------+------+-------

In [13]:
# Read the csv file
censo_pais = spark.read.format('csv').option("header", True).option("encoding", "utf-8").load(path + 'codes-cpais.csv')

print("First 10 rows of the CSV Country File:")
censo_pais.show(10)

First 10 rows of the CSV Country File:
+--------+------------+
|COD_PAIS|LITERAL_PAIS|
+--------+------------+
|     100|      EUROPA|
|     101|     ALBANIA|
|     102|     AUSTRIA|
|     103|     BELGICA|
|     104|    BULGARIA|
|     106|      CHIPRE|
|     107|   DINAMARCA|
|     108|      ESPAÑA|
|     109|   FINLANDIA|
|     110|     FRANCIA|
+--------+------------+
only showing top 10 rows



In [14]:
# Perform a join operation between the filtered DataFrame and the CSV file if the column CPAISN=COD_PAIS
censo_pro_country = censo_filtered.join(censo_pais, censo_filtered.CPAISN == censo_pais.COD_PAIS, how='left').drop(censo_pais.COD_PAIS)

print("First 10 rows of the joined DataFrame containing Province and Country name:")
censo_pro_country.show(10)

First 10 rows of the joined DataFrame containing Province and Country name:
+----+----+----------+------+--------------+----+----+----+----+----+------+-----+-----+------+----+----+----+------+-----+------+------------+----------+---------+-------+---------+---------+--------+--------+--------+-------+--------+--------+-------+----------+--------+------+-------+------+------+------+------+------+------+-----+------+----+-------+---+----+----+-----+------+------+------+------+---------+--------+--------+--------+------+------+------+-----+-----+----+----+------+--------+-------+---+----+-------+-------+--------+-------+------+--------+-------+------+------+---+-----+----+-----+-------+----------+----------+----------+----------+-------+------+-----+----+----+------+------+-------+----+-------+-----+-----+---+-----+-----+-----+-----+-----+----+-------+-------+-------+-------+--------+-------+----------+-------+-------+-------+-------+-------+-------+----------+-------+-------+-------+---

### 3. De las muestras que han quedado, las agrupamos por provincia de residencia y contamos cuántas personas hay en cada provincia

In [15]:
# Group by the column province of residence (CPRO) and count how many people are in each province
censo_grouped = censo_pro_country.groupBy('CPRO').count()

print("Number of people in each province:")
censo_grouped.show()

Number of people in each province:
+----+-----+
|CPRO|count|
+----+-----+
|  07|   47|
|  51|    2|
|  15|   45|
|  11|   33|
|  29|   57|
|  42|    4|
|  30|   79|
|  34|    7|
|  01|   13|
|  22|   20|
|  28|  201|
|  16|    5|
|  35|   16|
|  52|    2|
|  47|   30|
|  43|   29|
|  31|   67|
|  18|   47|
|  27|   19|
|  17|   79|
+----+-----+
only showing top 20 rows



### 4. Calculamos como resultado final la lista de las 10 provincias (incluyendo su nombre) con mayor número de personas procedentes o de Francia o de Portugal (la suma de ambos), y la ordenamos de mayor a menor

### 5. Recuperamos esa lista ordenada de 10 provincias junto con el número de inmigrantes de Francia y Portugal que ha obtenido, para mostrarla en pantalla.

In [21]:
# Select the column CPRO, NOMBRE_PRO, and count and sort the result by the count in descending order
censo_grouped = censo_pro_country.groupBy('CPRO', 'NOMBRE_PRO').count().sort(col('count').desc())

print("Number of people in each province with the name of the province:")
censo_grouped.show(10)

Number of people in each province with the name of the province:
+----+----------+-----+
|CPRO|NOMBRE_PRO|count|
+----+----------+-----+
|  28|    Madrid|  201|
|  08| Barcelona|  197|
|  46|  Valencia|  181|
|  03|  Alicante|   93|
|  30|    Murcia|   79|
|  17|    Girona|   79|
|  32|    Orense|   70|
|  36|Pontevedra|   68|
|  31|   Navarra|   67|
|  24|      León|   62|
+----+----------+-----+
only showing top 10 rows



In [20]:
# See how many people are from each country
censobycountry = censo_pro_country.groupBy('LITERAL_PAIS').count().sort(col('count').desc())

print("Number of people from each country:")
censobycountry.show()

Number of people from each country:
+------------+-----+
|LITERAL_PAIS|count|
+------------+-----+
|     FRANCIA| 1436|
|    PORTUGAL|  618|
+------------+-----+

